In [11]:
import numpy as np
from qutip import *

# --------------------------
# Platform scaling (pick one)
# --------------------------
PLATFORM = "ibm"   # "ibm" or "quera"

def platform_params(platform="ibm"):
    if platform.lower() == "ibm":
        # IBM-style transmon (rad/s, seconds)
        omega_max = 2*np.pi*12.5e6      # 40 ns pi pulse -> 12.5 MHz
        # T_max     = 0.32e-6             # ~8*pi/omega_max; use 1e-6..2e-6 for very adiabatic
        T_max     = 2e-6             # ~8*pi/omega_max; use 1e-6..2e-6 for very adiabatic
        Ep        = 2*np.pi*100e6       # penalty ~100 MHz
        time_unit_label = "time [µs]"
        energy_unit_label = "energy [MHz]"  # plots will convert rad/s -> MHz
        to_time_units = 1e6             # s -> µs
        to_freq_units = 1/(2*np.pi*1e6) # rad/s -> MHz
    elif platform.lower() == "quera":
        # QuEra Aquila (rad/µs, microseconds)
        omega_max = 10.0                # <= 15.8 rad/µs
        T_max     = 1.9                 # µs
        Ep        = 30.0                # rad/µs (>> omega_max)
        time_unit_label = "time [µs]"
        energy_unit_label = "energy [MHz]"  # convert rad/µs -> MHz
        to_time_units = 1.0             # µs -> µs
        to_freq_units = 1/(2*np.pi)     # rad/µs -> MHz
    else:
        raise ValueError("PLATFORM must be 'ibm' or 'quera'")
    return dict(omega_max=omega_max, T_max=T_max, Ep=Ep,
                time_unit_label=time_unit_label,
                energy_unit_label=energy_unit_label,
                to_time_units=to_time_units,
                to_freq_units=to_freq_units)

params = platform_params(PLATFORM)
omega_max = params["omega_max"]
T_max     = params["T_max"]
Ep        = params["Ep"]

# --------------------------
# Single-qubit primitives
# --------------------------
I = qeye(2)
X = sigmax()
Y = sigmay()
Z = sigmaz()

# --------------------------
# Logical operators (3-qubit repetition, bit-flip code)
# Code space: |0_L>=|000>, |1_L>=|111>
# Stabilizers for X-error protection: Z1Z2 and Z2Z3
# --------------------------
X_L = tensor(X, X, X)
Z_L = tensor(Z, Z, Z)              # acts nontrivially on code; anticommutes with X_L
I_L = tensor(I, I, I)

S1 = tensor(Z, Z, I)               # Z1 Z2
S2 = tensor(I, Z, Z)               # Z2 Z3
S_list = [S1, S2]

# Penalty Hamiltonian: -Ep * (S1 + S2)
Hp = -Ep * sum(S_list, 0*I_L)      # 'start' ensures Qobj sum

# --------------------------
# Basis & logical states (optional, for checks)
# --------------------------
basis_states = [basis(2, 0), basis(2, 1)]
logical_zero = tensor(basis_states[0], basis_states[0], basis_states[0])
logical_one  = tensor(basis_states[1], basis_states[1], basis_states[1])

# --------------------------
# RAP pulses
# --------------------------
def omega_t(t, T_max=T_max, omega_max=omega_max):
    return omega_max * np.sin(np.pi * t / T_max)

def delta_t(t, T_max=T_max, omega_max=omega_max):
    return -omega_max * np.cos(np.pi * t / T_max)

def H(t):
    # Logical RAP drive
    return X_L * omega_t(t) + Z_L * delta_t(t) + Hp

# --------------------------
# Time grid
# --------------------------
def time_list(num_points=101):
    return np.linspace(0, T_max, num_points)

t_list = time_list()

# --------------------------
# Spectrum vs. time with simple eigenstate matching
# --------------------------
def spectrum_track(t_list, n_track=2):
    """
    Track the lowest 'n_track' eigenstates by overlap with t=0 eigenstates.
    Returns: dict {idx: energies_over_time}
    """
    # t=0 diagonalization
    evals0, estates0 = H(0).eigenstates()
    dim = len(evals0)
    energies = {i: [evals0[i]] for i in range(dim)}

    for k in range(1, len(t_list)):
        t = t_list[k]
        evals, estates = H(t).eigenstates()

        # Track the first n_track states by overlap with initial estates0
        used = set()
        for i in range(n_track):
            ref = estates0[i]
            best_j, best_over = 0, -1.0
            for j, psi in enumerate(estates):
                if j in used:
                    continue
                ov = abs(ref.overlap(psi))**2
                if ov > best_over:
                    best_over, best_j = ov, j
            energies[i].append(evals[best_j])
            used.add(best_j)

        # Append the rest by index (no tracking)
        for i in range(n_track, dim):
            energies[i].append(evals[i])

    return energies, params

energies, p = spectrum_track(t_list, n_track=2)

# --------------------------
# Plot with Plotly (kept your style; kaleido optional)
# --------------------------
import plotly.graph_objects as go
import os

# Pulses
omega_vals = [omega_t(t) for t in t_list]
delta_vals = [delta_t(t) for t in t_list]

# Convert to nice plot units
t_plot   = t_list * p["to_time_units"]
w_plot   = np.array(omega_vals) * p["to_freq_units"]
d_plot   = np.array(delta_vals) * p["to_freq_units"]

os.makedirs('pngs', exist_ok=True)

fig = go.Figure()
fig.add_trace(go.Scatter(x=t_plot, y=w_plot, mode='lines', name='Ω(t)'))
fig.add_trace(go.Scatter(x=t_plot, y=d_plot, mode='lines', name='Δ(t)', line=dict(dash='dash')))
fig.update_layout(
    title=f"RAP pulses ({PLATFORM.upper()})",
    xaxis_title=p["time_unit_label"],
    yaxis_title="amplitude [MHz]",
    template="simple_white"
)
try:
    fig.write_image('pngs/omega_delta.png', scale=2, width=800, height=500)
except Exception as e:
    pass
fig.show()

# Spectrum figure
fig2 = go.Figure()
colors_main = ['red', 'blue']
for i, e_list in energies.items():
    e_plot = np.array(e_list) * p["to_freq_units"]
    if i < 2:
        fig2.add_trace(go.Scatter(x=t_plot, y=e_plot, mode='lines',
                                  line=dict(width=3, color=colors_main[i%2]),
                                  name=f"tracked state {i}"))
    else:
        fig2.add_trace(go.Scatter(x=t_plot, y=e_plot, mode='lines',
                                  line=dict(width=1, dash='dash'),
                                  name=f"state {i}"))
fig2.update_layout(
    title=f"Instantaneous spectrum vs time ({PLATFORM.upper()})",
    xaxis_title=p["time_unit_label"],
    yaxis_title=p["energy_unit_label"],
    template="simple_white",
    legend_title=""
)
try:
    fig2.write_image('pngs/spectrum.png', scale=2, width=900, height=600)
except Exception as e:
    pass
fig2.show()


In [12]:
# =========================
# Additions for open-system
# =========================
from qutip import sigmam, sigmap, Options, sesolve, mesolve

# --- Physical constants
kB   = 1.380649e-23        # J/K
hbar = 1.054571817e-34     # J*s

# Platform temperature for thermal factors
if "T_phys_K" not in params:
    params["T_phys_K"] = 0.015 if PLATFORM.lower() == "ibm" else 300.0

# ---------- Hamiltonians ----------
def H0(t):
    """Bare RAP logical Hamiltonian (no penalty)."""
    return X_L * omega_t(t) + Z_L * delta_t(t)

def make_H_func(Ep_val):
    """Return H(t, args) including the chosen penalty."""
    S_sum = S1 + S2
    def _H(t, args=None):
        return H0(t) - Ep_val * S_sum
    return _H

# ---------- Thermal factor (unit-consistent) ----------
def bose_einstein_N(omega_pos, T_K, platform=PLATFORM):
    if T_K <= 0 or omega_pos <= 0:
        return 0.0
    kBT_over_hbar_rad_per_s = (kB * T_K) / hbar      # [rad/s]
    kBT_over_hbar = kBT_over_hbar_rad_per_s * (1e-6 if platform.lower() == "quera" else 1.0)
    x = omega_pos / kBT_over_hbar
    x = np.clip(x, 1e-12, 700.0)
    return 1.0 / (np.exp(x) - 1.0)

def spectral_amp(omega_pos, lambda_2):
    return 0.0 if omega_pos <= 0 else np.sqrt(lambda_2)

# ---------- Collapse-operator generator ----------
def c_ops_gen_thermal(t_eval, H_func, n_qubits, lambda_2, T_K):
    Ht = H_func(t_eval, None)
    evals, evecs = Ht.eigenstates()
    dim = len(evals)

    c_ops = []
    for a in range(dim):
        for b in range(a+1, dim):
            omega = float(evals[b] - evals[a])  # >0
            if abs(omega) < 1e-12:
                continue

            rate_abs = 0.0  # a -> b
            rate_em  = 0.0  # b -> a
            for i in range(n_qubits):
                sigmam_i = tensor([sigmam() if j == i else I for j in range(n_qubits)])
                # complex scalar matrix element <b|sigmam_i|a>
                m_ba = (evecs[b].dag() * sigmam_i * evecs[a])
                mag2 = float(np.abs(m_ba)**2)
                if mag2 < 1e-16:
                    continue

                g = spectral_amp(omega, lambda_2)
                N = bose_einstein_N(omega, T_K)
                rate_abs += N       * (g**2) * mag2
                rate_em  += (N + 1) * (g**2) * mag2

            if rate_abs > 1e-16:
                Lop = (evecs[b] * evecs[a].dag())      # |b><a|
                c_ops.append(np.sqrt(rate_abs) * Lop)
            if rate_em > 1e-16:
                Lop = (evecs[a] * evecs[b].dag())      # |a><b|
                c_ops.append(np.sqrt(rate_em) * Lop)

    return c_ops

# ---------- Simulation setup ----------
n_qubits = 3
initial_state = logical_zero

rho0 = logical_zero * logical_zero.dag()
rho1 = logical_one  * logical_one.dag()
e_ops = [rho0, rho1]

# Ideal (unitary) benchmark with Ep=0
H_ideal = make_H_func(Ep_val=0.0)
res_ideal = sesolve(H_ideal, initial_state, t_list, e_ops=e_ops)
ideal_fidelity = res_ideal.expect[1]   # <1_L|rho|1_L>

# Sweep Ep values
gamma_sqrt = 0.1
lambda_2   = gamma_sqrt**2
T_K        = params["T_phys_K"]

Ep_values  = [0, 1, 2, 3, 4, 5, 10, 25, 50]  # same units as H

def _last_state(result):
    """Robustly fetch last state from a QuTiP result."""
    if hasattr(result, "states") and result.states:
        return result.states[-1]
    if hasattr(result, "state") and (result.state is not None):
        return result.state
    raise RuntimeError("Solver returned no states. Enable store_states=True.")

fidelity_curves = []
for Ep_val in Ep_values:
    print(f"Ep = {Ep_val}")
    H_fun = make_H_func(Ep_val)

    rho = initial_state * initial_state.dag()
    f_track = [float((rho1*rho).tr().real)]  # include t0

    for k in range(1, len(t_list)):
        t0, t1  = t_list[k-1], t_list[k]
        t_mid   = 0.5*(t0 + t1)
        c_ops   = c_ops_gen_thermal(t_mid, H_fun, n_qubits=n_qubits,
                                    lambda_2=lambda_2, T_K=T_K)

        opts = Options(store_states=True, nsteps=10000, rtol=1e-7, atol=1e-9)
        result = mesolve(H_fun, rho, [t0, t1], c_ops=c_ops,
                         e_ops=e_ops, args=None, options=opts)

        rho = _last_state(result)
        f_track.append(result.expect[1][-1])

    fidelity_curves.append(np.array(f_track))

# ---------- Plot ----------
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=t_list*params["to_time_units"], y=ideal_fidelity,
                         mode='lines', line=dict(dash='dash', width=3), name='Ideal (unitary)'))
for i, Ep_val in enumerate(Ep_values):
    fig.add_trace(go.Scatter(x=t_list*params["to_time_units"], y=fidelity_curves[i],
                             mode='lines', name=f"Ep={Ep_val}"))
fig.update_layout(
    title=f'Protocol fidelity vs time — {PLATFORM.upper()} units',
    xaxis_title=params["time_unit_label"],
    yaxis_title='Fidelity to |1_L⟩',
    template='plotly_white'
)
fig.show()

print("Done: Fidelity curves plotted.")


Ep = 0


/Users/chenscheim/miniconda3/lib/python3.12/site-packages/qutip/solver/options.py:16: FutureWarning:

Dedicated options class are no longer needed, options should be passed as dict to solvers.



Ep = 1
Ep = 2
Ep = 3
Ep = 4
Ep = 5
Ep = 10
Ep = 25
Ep = 50


Done: Fidelity curves plotted.
